# Introduction

We will use the python bindings for the [FANN](http://leenissen.dk/fann/wp/) (Fast Artificial Neural Networks) C-library to construct a neural network to map from the potential (represented by Fourier components, see [potentials.ipynb](potentials.ipynb)) to the eigenvalues (really, the difference between the eigenvalues and the square-well eigenvalues normalized by $\sqrt{\langle V_0^2 \rangle}$, see [eigenvalues.ipynb](eigenvalues.ipynb)). For FANN documentation, see [http://libfann.github.io/fann/docs/files/fann-h.html](http://libfann.github.io/fann/docs/files/fann-h.html).

# Preliminaries

In [8]:
%matplotlib inline
import numpy as np
from numpy.random import normal, randint
import matplotlib.pyplot as plt
from fann2 import libfann
from sklearn.cross_validation import train_test_split

# Number of basis states for the wavefunctions
NBW = 40
nbws = np.arange(1, NBW+1)
# Number of potentials:
NV = int(1E4)
# Number of basis states in the potential:
NB = 10
ns = np.arange(1, NB+1)
# lambda (variance of Legendre coefficients):
lam = 0.75
# The variance of the n=0 legendre coefficient V_0:
V20 = 10

# Input file:
filepath = "../Data/eigenvalues_NV" + str(NV) \
    + "_NB" + str(NB) + "_lam" \
    + str(lam) + "_V20" + str(V20) + ".npy"
filepathSD = "../Data/eigenvaluesSD_NV" + str(NV) \
    + "_NB" + str(NB) + "_lam" \
    + str(lam) + "_V20" + str(V20) + ".npy"
data = np.load(filepath)
dataSD = np.load(filepathSD)
VSns = data[::,0:10]
VCns = data[::,10:20]
eigs = data[::,20::]

In [9]:
print(data.shape, VSns.shape, VCns.shape, eigs.shape)
print(dataSD.shape)

(10000, 60) (10000, 10) (10000, 10) (10000, 40)
(40,)


# Preprocessing
We know that the spectrum is symmetric under $x\to -x$. We can build this into our dataset. We can duplicate the entire dataset but set all the Sine coefficients to their negative value. This effectively reflects the potential around the $y$-axis.

In [10]:
def VS(ns, xs):
    return np.sin(np.pi*np.outer(ns,xs))
def VC(ns, xs):
    return np.cos(np.pi*np.outer(ns,xs))
Nx = 100
xs = np.linspace(-1,1,Nx)
#Vgrid = legval(xs, np.transpose(Vns))
VSs = VS(ns,xs)
VCs = VC(ns,xs)

Vgrid = np.dot(VSns,VSs) + np.dot(VCns,VCs)
VgridFlipped = Vgrid[::,::-1]
print(Vgrid.shape)

(10000, 100)


In [11]:
numeigs = 5
X = np.concatenate( (np.concatenate((VSns, VCns), axis = 1), np.concatenate((-VSns, VCns), axis =1)) )
Xgrid = np.concatenate( (Vgrid, Vgrid[::,::-1]) )
y = np.concatenate( (eigs, eigs) )[::,1:numeigs+1]

In [12]:
# Split test and train
test_frac = 0.4
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_frac, random_state=7
)
test_frac = 0.4
X_train_grid, X_test_grid, y_train, y_test = train_test_split(
    Xgrid, y, test_size=test_frac, random_state=7
)

In [13]:
print(X.shape, y.shape)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print(Xgrid.shape, y.shape)
print(X_train_grid.shape, y_train.shape)
print(X_test_grid.shape, y_test.shape)

(20000, 20) (20000, 5)
(12000, 20) (12000, 5)
(8000, 20) (8000, 5)
(20000, 100) (20000, 5)
(12000, 100) (12000, 5)
(8000, 100) (8000, 5)


# Neural network

In [ ]:
# Model parameters
connection_rate = 1
learning_rate = 0.7
desired_error = 0.00001
max_iterations = 10000
iterations_between_reports = 100
num_train = X_train_grid.shape[0]
num_input = X_train_grid[1].size
num_output = numeigs
network = (num_input, 50,50,50, num_output)

# Initialize the neural network
ann = libfann.neural_net()
ann.create_sparse_array(connection_rate, network)
ann.set_learning_rate(learning_rate)
ann.set_activation_function_output(libfann.SIGMOID_SYMMETRIC_STEPWISE)

# Specify the training data
fann_train = libfann.training_data()
fann_train.set_train_data(X_train_grid.tolist(), y_train.tolist())

# Specify the test data
fann_test = libfann.training_data()
fann_test.set_train_data(X_test_grid.tolist(), y_test.tolist())

In [ ]:
# Training the network on the train data
ann.train_on_data(fann_train, max_iterations, iterations_between_reports, desired_error)

In [ ]:
# Testing the network on the test data
y_pred = []
for x in X_test:
    y_pred.append(ann.run(x))
y_pred = np.asarray(y_pred)
y_rel_err = np.abs((y_pred - y_test)/y_test)

In [ ]:
np.mean(y_rel_err, axis = 0)

In [ ]:
y_rel_err[3]

In [ ]:
def reject_outliers(data, iterations = 1, m=2, axis=0):
    out_data = data
    for i in range(0, iterations):
        indices = (abs(out_data - np.mean(out_data, axis)) < m * np.std(out_data, axis)).all(axis = 1)
        out_data = out_data[indices]
    return(out_data)

In [ ]:
y_err_out = reject_outliers(y_rel_err, iterations = 1, m=3)

In [ ]:
y_err_out.shape

In [ ]:
np.mean(y_err_out, axis = 0)